In [3]:
# Used for plotting data
%matplotlib inline
import matplotlib.pyplot as plt
# Used for data storage and manipulation 
import numpy as np
import pandas as pd
# Used for Regression Modelling
from sklearn.linear_model import LinearRegression
from sklearn import linear_model
from sklearn.model_selection import train_test_split
# Used for Acc metrics
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
# For stepwise regression
import statsmodels.api as sm
# box plots
import seaborn as sns
# pairplot
from seaborn import pairplot
# Correlation plot
from statsmodels.graphics.correlation import plot_corr
%autosave 60

C:\Users\jonth\Anaconda3\lib\importlib\_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)
C:\Users\jonth\Anaconda3\lib\importlib\_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)
C:\Users\jonth\Anaconda3\lib\importlib\_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)
C:\Users\jonth\Anaconda3\lib\importlib\_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)
C:\Users\jonth\Anaconda3\lib\importlib\_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 19

Autosaving every 60 seconds


In [138]:
# Load your data 
data = pd.read_csv("spreadspoke_scores.csv")
# Check out shape
print(data.shape)

#data from 2006-2018 is used for training. This will change, but is done this way for intial testing
no_2020 = data[(data['schedule_season'] < 2020)]
current = no_2020[(no_2020['schedule_season'] > 2005)]

# Adding .head() to your dataset allows you to see the first rows in the dataset. Input a number inside the brackets to specificy how many rows you want returned or else 5 rows are returned.
#test_data.head()
current.tail()

#adding a dictionary to convert team names to numbers
#numbers correspond to alphabetical order
team_dict = {}
team_dict['ARI'] = 0
team_dict['ATL'] = 1
team_dict['BAL'] = 2
team_dict['BUF'] = 3
team_dict['CAR'] = 4
team_dict['CHI'] = 5
team_dict['CIN'] = 6
team_dict['CLE'] = 7
team_dict['DAL'] = 8
team_dict['DEN'] = 9
team_dict['DET'] = 10
team_dict['GB'] = 11
team_dict['HOU'] = 12
team_dict['IND'] = 13
team_dict['JAX'] = 14
team_dict['KC'] = 15
team_dict['LAC'] = 16
team_dict['LAR'] = 17
team_dict['MIA'] = 18
team_dict['MIN'] = 19
team_dict['NE'] = 20
team_dict['NO'] = 21
team_dict['NYG'] = 22
team_dict['NYJ'] = 23
team_dict['LVR'] = 24
team_dict['PHI'] = 25
team_dict['PIT'] = 26
team_dict['SF'] = 27
team_dict['SEA'] = 28
team_dict['TB'] = 29
team_dict['TEN'] = 30
team_dict['WAS'] = 31
#needed to add PICK as it is a value for team_favorite_id. we can discard value by checking if values are equal to 99
team_dict['PICK'] = 99



(12934, 17)


In [149]:
clean_current = current[['schedule_season', 'team_home', 'score_home', 'score_away', 'team_away', 'team_favorite_id', 'spread_favorite', 'over_under_line', 'weather_temperature']]
clean_current.tail()

,schedule_season,team_home,score_home,score_away,team_away,team_favorite_id,spread_favorite,over_under_line,weather_temperature
12673,2019,Green Bay Packers,28.0,23.0,Seattle Seahawks,GB,-4.5,45.5,NaN
12674,2019,Kansas City Chiefs,51.0,31.0,Houston Texans,KC,-10.0,50.5,NaN
12675,2019,Kansas City Chiefs,35.0,24.0,Tennessee Titans,KC,-7.0,51,NaN
12676,2019,San Francisco 49ers,37.0,20.0,Green Bay Packers,SF,-8.0,46.5,NaN
12677,2019,Kansas City Chiefs,31.0,20.0,San Francisco 49ers,KC,-1.5,53,NaN


In [150]:
#print(sorted(clean_current["team_home"].unique()))
#print(sorted(clean_current["team_favorite_id"].unique()))

# doing the inane and cleaning the data so the team names match the abbreviations used in team_favorite_id
# this is for ease of comparison later on



for index, row in clean_current.iterrows():
    if row['team_home'] == "Arizona Cardinals":
        clean_current.at[index, 'team_home'] = "ARI"
        
    elif row['team_home'] == "Atlanta Falcons":
        clean_current.at[index, 'team_home'] = "ATL"
    elif row['team_home'] == "Baltimore Ravens":
        clean_current.at[index, 'team_home'] = "BAL"
    elif row['team_home'] == "Buffalo Bills":
        clean_current.at[index, 'team_home'] = "BUF"
    elif row['team_home'] == "Carolina Panthers":
        clean_current.at[index, 'team_home'] = "CAR"
    elif row['team_home'] == "Chicago Bears":
        clean_current.at[index, 'team_home'] = "CHI"
    elif row['team_home'] == "Cincinnati Bengals":
        clean_current.at[index, 'team_home'] = "CIN"
    elif row['team_home'] == "Cleveland Browns":
        clean_current.at[index, 'team_home'] = "CLE"
    elif row['team_home'] == "Dallas Cowboys":
        clean_current.at[index, 'team_home'] = "DAL"
    elif row['team_home'] == "Denver Broncos":
        clean_current.at[index, 'team_home'] = "DEN"
    elif row['team_home'] == "Detroit Lions":
        clean_current.at[index, 'team_home'] = "DET"
    elif row['team_home'] == "Green Bay Packers":
        clean_current.at[index, 'team_home'] = "GB"
    elif row['team_home'] == "Houston Texans":
        clean_current.at[index, 'team_home'] = "HOU"
    elif row['team_home'] == "Indianapolis Colts":
        clean_current.at[index, 'team_home'] = "IND"
    elif row['team_home'] == "Jacksonville Jaguars":
        clean_current.at[index, 'team_home'] = "JAX"
    elif row['team_home'] == "Kansas City Chiefs":
        clean_current.at[index, 'team_home'] = "KC"
    elif row['team_home'] == "Los Angeles Chargers" or row['team_home'] == "San Diego Chargers":
        clean_current.at[index, 'team_home'] = "LAC" # regardless of when team was in LA or SD, team ID is LAC
    elif row['team_home'] == "Los Angeles Rams" or row['team_home'] == "St. Louis Rams":
        clean_current.at[index, 'team_home'] = "LAR" # regardless of when team was in LA or STL, team ID is LAR
    elif row['team_home'] == "Miami Dolphins":
        clean_current.at[index, 'team_home'] = "MIA"
    elif row['team_home'] == "Minnesota Vikings":
        clean_current.at[index, 'team_home'] = "MIN"
    elif row['team_home'] == "New England Patriots":
        clean_current.at[index, 'team_home'] = "NE"
    elif row['team_home'] == "New Orleans Saints":
        clean_current.at[index, 'team_home'] = "NO"
    elif row['team_home'] == "New York Giants":
        clean_current.at[index, 'team_home'] = "NYG"
    elif row['team_home'] == "New York Jets":
        clean_current.at[index, 'team_home'] = "NYJ"
    elif row['team_home'] == "Oakland Raiders":
        clean_current.at[index, 'team_home'] = "LVR" # our data only has the Oakland Raiders, but team ID is still LVR b/c they're in Vegas now
    elif row['team_home'] == "Philadelphia Eagles":
        clean_current.at[index, 'team_home'] = "PHI"
    elif row['team_home'] == "Pittsburgh Steelers":
        clean_current.at[index, 'team_home'] = "PIT"
    elif row['team_home'] == "San Francisco 49ers":
        clean_current.at[index, 'team_home'] = "SF"
    elif row['team_home'] == "Seattle Seahawks":
        clean_current.at[index, 'team_home'] = "SEA"
    elif row['team_home'] == "Tampa Bay Buccaneers":
        clean_current.at[index, 'team_home'] = "TB"
    elif row['team_home'] == "Tennessee Titans":
        clean_current.at[index, 'team_home'] = "TEN"
    elif row['team_home'] == "Washington Redskins":
        clean_current.at[index, 'team_home'] = "WAS"
    # ------------------------------------------------------ now for away
    if row['team_away'] == "Arizona Cardinals":
        clean_current.at[index, 'team_away'] = "ARI"
    elif row['team_away'] == "Atlanta Falcons":
        clean_current.at[index, 'team_away'] = "ATL"
    elif row['team_away'] == "Baltimore Ravens":
        clean_current.at[index, 'team_away'] = "BAL"
    elif row['team_away'] == "Buffalo Bills":
        clean_current.at[index, 'team_away'] = "BUF"
    elif row['team_away'] == "Carolina Panthers":
        clean_current.at[index, 'team_away'] = "CAR"
    elif row['team_away'] == "Chicago Bears":
        clean_current.at[index, 'team_away'] = "CHI"
    elif row['team_away'] == "Cincinnati Bengals":
        clean_current.at[index, 'team_away'] = "CIN"
    elif row['team_away'] == "Cleveland Browns":
        clean_current.at[index, 'team_away'] = "CLE"
    elif row['team_away'] == "Dallas Cowboys":
        clean_current.at[index, 'team_away'] = "DAL"
    elif row['team_away'] == "Denver Broncos":
        clean_current.at[index, 'team_away'] = "DEN"
    elif row['team_away'] == "Detroit Lions":
        clean_current.at[index, 'team_away'] = "DET"
    elif row['team_away'] == "Green Bay Packers":
        clean_current.at[index, 'team_away'] = "GB"
    elif row['team_away'] == "Houston Texans":
        clean_current.at[index, 'team_away'] = "HOU"
    elif row['team_away'] == "Indianapolis Colts":
        clean_current.at[index, 'team_away'] = "IND"
    elif row['team_away'] == "Jacksonville Jaguars":
        clean_current.at[index, 'team_away'] = "JAX"
    elif row['team_away'] == "Kansas City Chiefs":
        clean_current.at[index, 'team_away'] = "KC"
    elif row['team_away'] == "Los Angeles Chargers" or row['team_away'] == "San Diego Chargers":
        clean_current.at[index, 'team_away'] = "LAC" # regardless of when team was in LA or SD, team ID is LAC
    elif row['team_away'] == "Los Angeles Rams" or row['team_away'] == "St. Louis Rams":
        clean_current.at[index, 'team_away'] = "LAR" # regardless of when team was in LA or STL, team ID is LAR
    elif row['team_away'] == "Miami Dolphins":
        clean_current.at[index, 'team_away'] = "MIA"
    elif row['team_away'] == "Minnesota Vikings":
        clean_current.at[index, 'team_away'] = "MIN"
    elif row['team_away'] == "New England Patriots":
        clean_current.at[index, 'team_away'] = "NE"
    elif row['team_away'] == "New Orleans Saints":
        clean_current.at[index, 'team_away'] = "NO"
    elif row['team_away'] == "New York Giants":
        clean_current.at[index, 'team_away'] = "NYG"
    elif row['team_away'] == "New York Jets":
        clean_current.at[index, 'team_away'] = "NYJ"
    elif row['team_away'] == "Oakland Raiders":
        clean_current.at[index, 'team_away'] = "LVR" # our data only has the Oakland Raiders, but team ID is still LVR b/c they're in Vegas now
    elif row['team_away'] == "Philadelphia Eagles":
        clean_current.at[index, 'team_away'] = "PHI"
    elif row['team_away'] == "Pittsburgh Steelers":
        clean_current.at[index, 'team_away'] = "PIT"
    elif row['team_away'] == "San Francisco 49ers":
        clean_current.at[index, 'team_away'] = "SF"
    elif row['team_away'] == "Seattle Seahawks":
        clean_current.at[index, 'team_away'] = "SEA"
    elif row['team_away'] == "Tampa Bay Buccaneers":
        clean_current.at[index, 'team_away'] = "TB"
    elif row['team_away'] == "Tennessee Titans":
        clean_current.at[index, 'team_away'] = "TEN"
    elif row['team_away'] == "Washington Redskins":
        clean_current.at[index, 'team_away'] = "WAS"
# damn that was a lot, wonder if that can be simplified somehow lmao

clean_current.head()
# --------------------------------------------------------

,schedule_season,team_home,score_home,score_away,team_away,team_favorite_id,spread_favorite,over_under_line,weather_temperature
8940,2006,PIT,28.0,17.0,MIA,PIT,-1.5,34.5,62.0
8941,2006,ARI,34.0,27.0,SF,ARI,-9.5,44,72.0
8942,2006,CAR,6.0,20.0,ATL,CAR,-4.5,39,73.0
8943,2006,CLE,14.0,19.0,NO,CLE,-3.0,36,65.0
8944,2006,DET,6.0,9.0,SEA,SEA,-6.0,44,72.0


In [151]:
#have to change spread_favorite to home_spread


for index, row in clean_current.iterrows():
    #spread favorite to home_spread
    if row['team_favorite_id'] == row['team_away']:
        clean_current.at[index, 'spread_favorite'] = clean_current.at[index, 'spread_favorite']*-1

clean_current.tail(20)

,schedule_season,team_home,score_home,score_away,team_away,team_favorite_id,spread_favorite,over_under_line,weather_temperature
12658,2019,HOU,14.0,35.0,TEN,TEN,10.0,43,72.0
12659,2019,JAX,38.0,20.0,IND,IND,5.5,41.5,NaN
12660,2019,KC,31.0,21.0,LAC,KC,-9.5,46,NaN
12661,2019,LAR,31.0,24.0,ARI,LAR,-7.0,45.5,NaN
12662,2019,MIN,19.0,21.0,CHI,CHI,4.5,36,72.0
12663,2019,NE,24.0,27.0,MIA,NE,-17.0,45.5,NaN
12664,2019,NYG,17.0,34.0,PHI,PHI,3.5,44.5,NaN
12665,2019,SEA,21.0,26.0,SF,SF,3.5,47,NaN
12666,2019,TB,22.0,28.0,ATL,TB,-1.0,48.5,NaN
12667,2019,HOU,22.0,19.0,BUF,HOU,-2.5,43,72.0


In [146]:
#Johnny's cell for logistic regression attempt. will use clean_current as data. 
#linear regression imported at first cell so no need to import here
#this cell actuallly works
#this model predicts a winnner, given the scores
#wow, its 100% accurate when we give it the score and tell it to predict who wins
from sklearn.linear_model import LogisticRegression

#for intial modeling I am using the 2019 season as test data
test_data = clean_current[(clean_current['schedule_season'] > 2018)]

#remove 2019 season from training data
features_list = clean_current[(clean_current['schedule_season'] < 2018)]
    
y_train = []
y_test = []

x_train = features_list[['team_home', 'score_home', 'score_away', 'team_away', 
                               'team_favorite_id', 'spread_favorite', 'over_under_line']]

x_test = test_data[['team_home', 'score_home', 'score_away', 'team_away', 
                                           'team_favorite_id', 'spread_favorite', 'over_under_line']]
#convert team_name strings to numbers
for index, row in x_train.iterrows():
    x_train.at[index, 'team_away'] = team_dict[row['team_away']]
    x_train.at[index, 'team_home'] = team_dict[row['team_home']]
    x_train.at[index, 'team_favorite_id'] = team_dict[row['team_favorite_id']]

#same for test
#convert team_name strings to numbers
for index, row in x_test.iterrows():
    x_test.at[index, 'team_away'] = team_dict[row['team_away']]
    x_test.at[index, 'team_home'] = team_dict[row['team_home']]
    x_test.at[index, 'team_favorite_id'] = team_dict[row['team_favorite_id']]

#TRAINING DATA
#get array y with winners of the games
#1 if home team wins, 0 if away team wins
for index, row in x_train.iterrows():
    if row['score_home'] > row['score_away']:
        #np.append(y,val[1])
        y_train.append(1)
    else:
        #np.append(y,val[4])
        y_train.append(0)
        
#TEST DATA
#get array y with winners of the games
#1 if home team wins, 0 if away team wins
for index, row in x_test.iterrows():
    if row['score_home'] > row['score_away']:
        #np.append(y,val[1])
        y_test.append(1)
    else:
        #np.append(y,val[4])
        y_test.append(0)
        
y_train = np.array(y_train)
y_test = np.array(y_test)

#x is array of arrays. each element of x is an array with data for a game
x_train = x_train.to_numpy()
x_test = x_test.to_numpy()


model = LogisticRegression(solver = 'lbfgs')

model.fit(x_train,y_train)

y_pred = model.predict(x_test)

print('predicted response:', y_pred, sep='\n')
print('actual response:', y_test, sep='\n')

score = model.score(x_test, y_test)
print(score)

predicted response:
[0 0 0 0 1 0 1 0 1 1 0 1 1 0 1 1 0 1 1 0 0 1 1 1 1 0 0 0 0 0 0 0 1 0 1 0 1
 1 1 1 0 1 1 0 1 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 1 1 1 1 1 0 0 1 0 0 1 0 1 1
 0 0 0 1 1 1 1 0 1 0 0 0 0 0 1 1 0 1 0 0 1 0 0 1 0 1 1 0 0 1 0 0 1 0 0 0 1
 1 1 1 0 1 1 1 1 1 1 0 1 1 1 1 0 1 1 1 1 1 1 1 0 1 1 0 1 0 1 0 0 1 1 1 1 0
 1 1 0 0 1 1 0 1 1 0 1 0 0 0 1 0 1 1 0 1 1 1 1 0 1 1 1 0 0 0 0 0 1 0 1 1 1
 0 0 1 1 0 1 1 1 0 1 0 1 1 0 0 1 1 0 0 1 0 1 1 1 1 0 0 1 0 1 1 0 1 0 0 0 0
 0 1 1 1 0 1 0 0 1 1 0 1 1 1 0 0 0 0 1 0 0 1 1 1 0 0 1 1 1 0 0 0 0 0 1 0 0
 0 0 1 1 1 1 1 1]
actual response:
[0 0 0 0 1 0 1 0 1 1 0 1 1 0 1 1 0 1 1 0 0 1 1 1 1 0 0 0 0 0 0 0 1 0 1 0 1
 1 1 1 0 1 1 0 1 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 1 1 1 1 1 0 0 1 0 0 1 0 1 1
 0 0 0 1 1 1 1 0 1 0 0 0 0 0 1 1 0 1 0 0 1 0 0 1 0 1 1 0 0 1 0 0 1 0 0 0 1
 1 1 1 0 1 1 1 1 1 1 0 1 1 1 1 0 1 1 1 1 1 1 1 0 1 1 0 1 0 1 0 0 1 1 1 1 0
 1 1 0 0 1 1 0 1 1 0 1 0 0 0 1 0 1 1 0 1 1 1 1 0 1 1 1 0 0 0 0 0 1 0 1 1 1
 0 0 1 1 0 1 1 1 0 1 0 1 1 0 0 1 1 0 0 1 0 1 

C:\Users\jonth\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


In [169]:
#Johnny's cell for logistic regression attempt. will use clean_current as data. 
from sklearn.linear_model import LogisticRegression
from sklearn import metrics as mt
#for intial modeling I am using the 2019-2020 season as test data
test_data = clean_current[(clean_current['schedule_season'] > 2018)]

#remove 2019 season from training data
features_list = clean_current[(clean_current['schedule_season'] < 2018)]
    
y_train = []
y_test = []

x_train = features_list[['team_home', 'score_home', 'score_away', 'team_away', 
                               'team_favorite_id', 'spread_favorite', 'over_under_line', 'weather_temperature']]

x_test = test_data[['team_home', 'score_home', 'score_away', 'team_away', 
                                           'team_favorite_id', 'spread_favorite', 'over_under_line', 'weather_temperature']]
#convert team_name strings to numbers
for index, row in x_train.iterrows():
    x_train.at[index, 'team_away'] = team_dict[row['team_away']]
    x_train.at[index, 'team_home'] = team_dict[row['team_home']]
    x_train.at[index, 'team_favorite_id'] = team_dict[row['team_favorite_id']]

#same for test
#convert team_name strings to numbers
for index, row in x_test.iterrows():
    x_test.at[index, 'team_away'] = team_dict[row['team_away']]
    x_test.at[index, 'team_home'] = team_dict[row['team_home']]
    x_test.at[index, 'team_favorite_id'] = team_dict[row['team_favorite_id']]

#TRAINING DATA
#get array y with winners of the games
#1 if home team wins, 0 if away team wins
for index, row in x_train.iterrows():
    if row['score_home'] > row['score_away']:
        #np.append(y,val[1])
        y_train.append(1)
    else:
        #np.append(y,val[4])
        y_train.append(0)
        
#TEST DATA
#get array y with winners of the games
#1 if home team wins, 0 if away team wins
for index, row in x_test.iterrows():
    if row['score_home'] > row['score_away']:
        #np.append(y,val[1])
        y_test.append(1)
    else:
        #np.append(y,val[4])
        y_test.append(0)

x_train = x_train[['spread_favorite']]
x_test = x_test[['spread_favorite']]

y = np.array(y)
y_test = np.array(y_test)

#x is array of arrays. each element of x is an array with data for a game
x_train = x_train.to_numpy()
x_test = x_test.to_numpy()

#convert over_under to float
#for val in x_train:
#    val[1] = float(val[1])
    
model = LogisticRegression(solver = 'lbfgs')

model.fit(x_train,y_train)

y_pred = model.predict(x_test)
print('predicted outcome:', y_pred, sep='\n')
print('actual outcome:', y_test, sep='\n')

score = mt.accuracy_score(y_test, y_pred)
conf = mt.confusion_matrix(y_test, y_pred)
print(score)
score2 = model.score(x_test, y_test)
print('score_other_way\n', score2)
print('len of x_train\n',len(x_train))
print('len of x_test\n',len(x_test))
print('confusion matrix\n',conf)

predicted outcome:
[1 0 0 1 1 0 1 0 1 1 1 1 1 1 1 0 1 0 1 1 0 0 1 1 1 0 0 0 1 1 0 0 0 1 1 0 1
 1 1 1 1 1 1 1 1 1 1 0 1 0 1 1 0 1 1 0 1 1 1 0 0 1 1 1 1 1 1 1 1 1 1 0 0 1
 0 1 0 1 1 0 1 1 1 1 1 1 1 0 1 0 0 1 0 0 1 1 0 1 0 1 1 1 1 1 0 0 1 0 1 1 1
 1 1 1 0 1 1 1 1 1 1 0 0 1 1 0 1 0 0 0 1 1 1 1 0 0 1 0 1 1 1 1 1 0 0 1 0 1
 1 1 1 0 1 1 0 1 1 0 1 0 1 0 1 1 1 1 0 1 1 1 0 0 1 1 0 0 0 1 0 0 1 1 0 0 0
 0 0 1 0 0 0 1 0 0 1 0 1 1 1 0 1 1 1 1 1 0 1 1 1 0 0 0 0 0 1 1 0 1 1 1 1 1
 0 1 1 1 0 1 0 0 1 1 1 0 0 0 1 0 0 1 0 0 0 0 1 1 0 0 0 1 1 0 1 0 0 1 1 1 1
 1 1 1 1 1 1 1 1]
actual outcome:
[0 0 0 0 1 0 1 0 1 1 0 1 1 0 1 1 0 1 1 0 0 1 1 1 1 0 0 0 0 0 0 0 1 0 1 0 1
 1 1 1 0 1 1 0 1 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 1 1 1 1 1 0 0 1 0 0 1 0 1 1
 0 0 0 1 1 1 1 0 1 0 0 0 0 0 1 1 0 1 0 0 1 0 0 1 0 1 1 0 0 1 0 0 1 0 0 0 1
 1 1 1 0 1 1 1 1 1 1 0 1 1 1 1 0 1 1 1 1 1 1 1 0 1 1 0 1 0 1 0 0 1 1 1 1 0
 1 1 0 0 1 1 0 1 1 0 1 0 0 0 1 0 1 1 0 1 1 1 1 0 1 1 1 0 0 0 0 0 1 0 1 1 1
 0 0 1 1 0 1 1 1 0 1 0 1 1 0 0 1 1 0 0 1 0 1 1 

In [ ]:
# Tanner's cells for using linear regression to predict what Vegas put down as their spread and over/under.

# we will focus on regular season statistics.
regszn = current[(current['schedule_playoff'] == False)]
regszn = regszn[['schedule_season', 'schedule_week', 'team_home', 'score_home', 'score_away', 'team_away', 
                 'team_favorite_id', 'spread_favorite', 'over_under_line', 'weather_temperature', 'weather_wind_mph']]

# I want to predict what Vegas put down as the spread and the O/U.
# My thinking is that, from the data we have that would be available pre-game, Vegas would look at:
#  * who's playing, who's the favorite, and who is home v. away
#  * the week of the season
#  * the temperature
#  * the wind
# More than likely, this will not be a great predictor. For a better predictor we would also need extensive 
#    week-by-week historical stats for each team, which is not exactly easy to do, especially when this 
#    project is due on monday and you have other projects and exams to study for.
# for some reason the week and O/U in the dataset aren't numerical. Let's fix that.
regszn['schedule_week'] = regszn['schedule_week'].astype(float)
regszn['over_under_line'] = regszn['over_under_line'].astype(float)
convertNamesToIndices(regszn)

# putting together our dataframes for the modeling.
X = pd.DataFrame(regszn, columns = ['schedule_week', 'team_home', 'team_away', 'team_favorite_id', 
                                    'weather_temperature', 'weather_wind_mph'])
spread_y = pd.DataFrame(regszn, columns = ['spread_favorite'])
OU_y = pd.DataFrame(regszn, columns = ['over_under_line'])

In [ ]:
# spread model
spread_X_train, spread_X_test, spread_y_train, spread_y_test = train_test_split(X, spread_y, random_state = 1)
spread_lm = LinearRegression()
spread_lm.fit(spread_X_train, spread_y_train)
spread_pred = spread_lm.predict(spread_X_test)

spread_X_test['predicted_spread'] = np.around(spread_pred, 1)
spread_X_test['actual_vegas_spread'] = spread_y_test
spread_X_test['spread_difference'] = np.around(abs(spread_y_test - spread_pred), 1)

spread_X_test

In [ ]:
print("Average Difference between our spread and Vegas' spread:", np.mean(spread_X_test['spread_difference']))

spread_test_r2 = r2_score(spread_y_test, spread_pred)
print("R2 Score of Spread Model:", spread_test_r2)

spread_test_rmse = (np.sqrt(mean_squared_error(spread_y_test, spread_pred)))
print("RMSE of Spread Model:", spread_test_rmse)

spread_corr = spread_X_test.corr()
spread_corr

In [ ]:
# over/under model
OU_X_train, OU_X_test, OU_y_train, OU_y_test = train_test_split(X, OU_y, random_state = 1)
OU_lm = LinearRegression()
OU_lm.fit(OU_X_train, OU_y_train)
OU_pred = OU_lm.predict(OU_X_test)

OU_X_test['predicted_over_under'] = np.around(OU_pred, 1)
OU_X_test['actual_over_under_line'] = OU_y_test
OU_X_test['ou_difference'] = np.around(abs(OU_y_test - OU_pred), 1)

OU_X_test

In [ ]:
print("Average Difference between our O/U and Vegas' O/U:", np.mean(OU_X_test['ou_difference']))

OU_test_r2 = r2_score(OU_y_test, OU_pred)
print("R2 Score of O/U Model:", OU_test_r2)

OU_test_rmse = (np.sqrt(mean_squared_error(OU_y_test, OU_pred)))
print("RMSE of O/U Model:", OU_test_rmse)

OU_corr = OU_X_test.corr()
OU_corr

SyntaxError: invalid syntax (<ipython-input-170-d0fd8377ff8d>, line 1)